## Modeling Survival on the Titanic

In [1]:
import pandas as pd
import numpy as np
import re
import pickle

In [2]:
import plotly as py
import plotly.graph_objs as go

In [3]:
from sklearn import svm
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [4]:
# Define the color palette 
Viridis=[
"#440154", "#440558", "#450a5c", "#450e60", "#451465", "#461969",
"#461d6d", "#462372", "#472775", "#472c7a", "#46307c", "#45337d",
"#433880", "#423c81", "#404184", "#3f4686", "#3d4a88", "#3c4f8a",
"#3b518b", "#39558b", "#37598c", "#365c8c", "#34608c", "#33638d",
"#31678d", "#2f6b8d", "#2d6e8e", "#2c718e", "#2b748e", "#29788e",
"#287c8e", "#277f8e", "#25848d", "#24878d", "#238b8d", "#218f8d",
"#21918d", "#22958b", "#23988a", "#239b89", "#249f87", "#25a186",
"#25a584", "#26a883", "#27ab82", "#29ae80", "#2eb17d", "#35b479",
"#3cb875", "#42bb72", "#49be6e", "#4ec16b", "#55c467", "#5cc863",
"#61c960", "#6bcc5a", "#72ce55", "#7cd04f", "#85d349", "#8dd544",
"#97d73e", "#9ed93a", "#a8db34", "#b0dd31", "#b8de30", "#c3df2e",
"#cbe02d", "#d6e22b", "#e1e329", "#eae428", "#f5e626", "#fde725"]
# source: https://bhaskarvk.github.io/colormap/reference/colormap.html

### Read in the titanic dataset

In [5]:
df = pd.read_csv('resources/titanic.csv')
# df = pd.read_csv("https://raw.githubusercontent.com/austinlasseter/plotly_dash_tutorial/master/00%20resources/titanic.csv")
print(df.shape)
df.head()

(712, 8)


,Survived,Pclass,Sex,Age,Fare,Embarked,Name,SibSp
0,0,3,male,22.0,7.2500,Southampton,"Braund, Mr. Owen Harris",1
1,1,1,female,38.0,71.2833,Cherbourg,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1
2,1,3,female,26.0,7.9250,Southampton,"Heikkinen, Miss. Laina",0
3,1,1,female,35.0,53.1000,Southampton,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1
4,0,3,male,35.0,8.0500,Southampton,"Allen, Mr. William Henry",0


### Features

In [6]:
# Make some dummies for sex.
df = pd.get_dummies(df, prefix='', prefix_sep='', columns=['Sex'])

In [7]:
# Make some dummies Passenger's cabin class
df = pd.get_dummies(df, prefix='Cabin Class', prefix_sep=' ', columns=['Pclass'])

In [8]:
# Make some dummies Passenger's embarkation
df = pd.get_dummies(df, prefix='', prefix_sep='', columns=['Embarked'])

In [9]:
# Siblings and Spouses
df=df.rename(columns={'SibSp':'Siblings and Spouses'})
df['Siblings and Spouses'].value_counts()

0    469
1    183
2     25
4     18
3     12
5      5
Name: Siblings and Spouses, dtype: int64

In [10]:
# age requires binning
print(df.Age.describe())
print(df.Age.describe()['25%'])

count    712.000000
mean      29.642093
std       14.492933
min        0.420000
25%       20.000000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64
20.0


In [11]:
# Age
bins=[1, 20, 28, 38, 80]
df['age_binned']=pd.cut(df['Age'], bins)
df['age_binned'].value_counts().sort_index()

(1, 20]     165
(20, 28]    183
(28, 38]    174
(38, 80]    176
Name: age_binned, dtype: int64

In [12]:
df = pd.get_dummies(df, prefix='Age', prefix_sep=' ', columns=['age_binned'])
df.head(2)

,Survived,Age,Fare,Name,Siblings and Spouses,female,male,Cabin Class 1,Cabin Class 2,Cabin Class 3,Cherbourg,Queenstown,Southampton,"Age (1, 20]","Age (20, 28]","Age (28, 38]","Age (38, 80]"
0,0,22.0,7.2500,"Braund, Mr. Owen Harris",1,0,1,0,0,1,0,0,1,0,1,0,0
1,1,38.0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1,0,1,0,0,1,0,0,0,0,1,0


In [13]:
# Name
df['Last Name']=df['Name'].apply(lambda x: x.split(',')[0])
df['First Name']=df['Name'].apply(lambda x: x.split(',')[1])
df['Title']=df['First Name'].apply(lambda x: x.split('.')[0])

In [14]:
# Title
print(df['Title'].value_counts())
df['Mr.']=np.where((df['Title']==' Mr')|(df['Title']==' Master')|(df['Title']==' Ms'), 1, 0) # Ms is actually monsieur
df['Mrs.']=np.where((df['Title']==' Mrs')|(df['Title']==' Mme'), 1, 0)
df['Miss']=np.where((df['Title']==' Miss')|(df['Title']==' Mlle'), 1, 0)
df['VIP']=np.where((df['Mr.']==0)&(df['Mrs.']==0)&(df['Miss']==0), 1, 0)

 Mr              398
 Miss            145
 Mrs             107
 Master           36
 Rev               6
 Dr                6
 Mlle              2
 Major             2
 Col               2
 the Countess      1
 Capt              1
 Ms                1
 Sir               1
 Lady              1
 Mme               1
 Don               1
 Jonkheer          1
Name: Title, dtype: int64


In [16]:
df['VIP'].value_counts()

0    690
1     22
Name: VIP, dtype: int64

In [17]:
df.isna().sum()

Survived                0
Age                     0
Fare                    0
Name                    0
Siblings and Spouses    0
female                  0
male                    0
Cabin Class 1           0
Cabin Class 2           0
Cabin Class 3           0
Cherbourg               0
Queenstown              0
Southampton             0
Age (1, 20]             0
Age (20, 28]            0
Age (28, 38]            0
Age (38, 80]            0
Last Name               0
First Name              0
Title                   0
Mr.                     0
Mrs.                    0
Miss                    0
VIP                     0
dtype: int64

In [16]:
# Check for missing values as they will skew the regression
print(df.shape)
df = df.dropna()
print(df.shape)

(712, 24)
(712, 24)


In [18]:
# What are the possible features?
df.columns

Index(['Survived', 'Age', 'Fare', 'Name', 'Siblings and Spouses', 'female',
       'male', 'Cabin Class 1', 'Cabin Class 2', 'Cabin Class 3', 'Cherbourg',
       'Queenstown', 'Southampton', 'Age (1, 20]', 'Age (20, 28]',
       'Age (28, 38]', 'Age (38, 80]', 'Last Name', 'First Name', 'Title',
       'Mr.', 'Mrs.', 'Miss', 'VIP'],
      dtype='object')

In [20]:
df['Title'].unique()

array([' Mr', ' Mrs', ' Miss', ' Master', ' Don', ' Rev', ' Dr', ' Mme',
       ' Ms', ' Major', ' Lady', ' Sir', ' Mlle', ' Col', ' Capt',
       ' the Countess', ' Jonkheer'], dtype=object)

In [32]:
a = df['Fare'].unique()
a.sort()
a

array([  0.    ,   4.0125,   5.    ,   6.2375,   6.4375,   6.45  ,
         6.4958,   6.75  ,   6.975 ,   7.0458,   7.05  ,   7.0542,
         7.125 ,   7.1417,   7.225 ,   7.2292,   7.25  ,   7.4958,
         7.5208,   7.55  ,   7.65  ,   7.7333,   7.7417,   7.75  ,
         7.775 ,   7.7958,   7.8   ,   7.8542,   7.875 ,   7.8792,
         7.8875,   7.8958,   7.925 ,   8.0292,   8.05  ,   8.1583,
         8.3   ,   8.3625,   8.4042,   8.4333,   8.5167,   8.6542,
         8.6625,   8.6833,   8.85  ,   9.    ,   9.2167,   9.225 ,
         9.35  ,   9.475 ,   9.4833,   9.5   ,   9.5875,   9.825 ,
         9.8375,   9.8417,   9.8458,  10.1708,  10.4625,  10.5   ,
        10.5167,  11.1333,  11.2417,  11.5   ,  12.    ,  12.275 ,
        12.2875,  12.35  ,  12.475 ,  12.525 ,  12.65  ,  12.875 ,
        13.    ,  13.4167,  13.5   ,  13.7917,  13.8583,  14.    ,
        14.1083,  14.4   ,  14.4542,  14.4583,  14.5   ,  15.    ,
        15.0458,  15.2458,  15.5   ,  15.55  ,  15.7417,  15.7

In [33]:
print(df[df['Fare'] > 512])

     Survived   Age      Fare                                Name  \
207         1  35.0  512.3292                    Ward, Miss. Anna   
536         1  36.0  512.3292  Cardeza, Mr. Thomas Drake Martinez   
586         1  35.0  512.3292              Lesurer, Mr. Gustave J   

     Siblings and Spouses  female  male  Cabin Class 1  Cabin Class 2  \
207                     0       1     0              1              0   
536                     0       0     1              1              0   
586                     0       0     1              1              0   

     Cabin Class 3  ...  Age (20, 28]  Age (28, 38]  Age (38, 80]  Last Name  \
207              0  ...             0             1             0       Ward   
536              0  ...             0             1             0    Cardeza   
586              0  ...             0             1             0    Lesurer   

                     First Name  Title  Mr. Mrs. Miss VIP  
207                  Miss. Anna   Miss    0    0 

In [36]:
bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120,  140, 150, 160, 170,  220,  240, 250,  270, 600]
df['fare_binned']=pd.cut(df['Fare'], bins)
df['fare_binned'].value_counts().sort_index()

(0, 10]       229
(10, 20]      155
(20, 30]      120
(30, 40]       51
(40, 50]       14
(50, 60]       31
(60, 70]       10
(70, 80]       28
(80, 90]       15
(90, 100]       4
(100, 110]      4
(110, 120]     10
(120, 140]      6
(140, 150]      1
(150, 160]      7
(160, 170]      2
(170, 220]      4
(220, 240]      3
(240, 250]      2
(250, 270]      6
(270, 600]      3
Name: fare_binned, dtype: int64

In [37]:
df = pd.get_dummies(df, prefix='Fare', prefix_sep=' ', columns=['fare_binned'])
df

,Survived,Age,Fare,Name,Siblings and Spouses,female,male,Cabin Class 1,Cabin Class 2,Cabin Class 3,...,"Fare (110, 120]","Fare (120, 140]","Fare (140, 150]","Fare (150, 160]","Fare (160, 170]","Fare (170, 220]","Fare (220, 240]","Fare (240, 250]","Fare (250, 270]","Fare (270, 600]"
0,0,22.0,7.2500,"Braund, Mr. Owen Harris",1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,38.0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,26.0,7.9250,"Heikkinen, Miss. Laina",0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,35.0,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,35.0,8.0500,"Allen, Mr. William Henry",0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,0,39.0,29.1250,"Rice, Mrs. William (Margaret Norton)",0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
708,0,27.0,13.0000,"Montvila, Rev. Juozas",0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
709,1,19.0,30.0000,"Graham, Miss. Margaret Edith",0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
710,1,26.0,30.0000,"Behr, Mr. Karl Howell",0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# Turns out that fare doesn't add any significant or meaningful coefficient to the final model, so we're dropping it.
feature_cols=list(df.columns)
for item in ['Survived', 'Fare', 'Cabin Class 1', 'Southampton', 'male', 'Age',  'Age (1, 20]', 'Last Name', 'First Name', 'Title', 'Mr.']:
    feature_cols.remove(item)
print(feature_cols)

['Name', 'Siblings and Spouses', 'female', 'Cabin Class 2', 'Cabin Class 3', 'Cherbourg', 'Queenstown', 'Age (20, 28]', 'Age (28, 38]', 'Age (38, 80]', 'Mrs.', 'Miss', 'VIP', 'Fare (0, 10]', 'Fare (10, 20]', 'Fare (20, 30]', 'Fare (30, 40]', 'Fare (40, 50]', 'Fare (50, 60]', 'Fare (60, 70]', 'Fare (70, 80]', 'Fare (80, 90]', 'Fare (90, 100]', 'Fare (100, 110]', 'Fare (110, 120]', 'Fare (120, 140]', 'Fare (140, 150]', 'Fare (150, 160]', 'Fare (160, 170]', 'Fare (170, 220]', 'Fare (220, 240]', 'Fare (240, 250]', 'Fare (250, 270]', 'Fare (270, 600]']


In [39]:
# Select our features
X = df[feature_cols]
y = df.Survived

In [42]:
X

,Name,Siblings and Spouses,female,Cabin Class 2,Cabin Class 3,Cherbourg,Queenstown,"Age (20, 28]","Age (28, 38]","Age (38, 80]",...,"Fare (110, 120]","Fare (120, 140]","Fare (140, 150]","Fare (150, 160]","Fare (160, 170]","Fare (170, 220]","Fare (220, 240]","Fare (240, 250]","Fare (250, 270]","Fare (270, 600]"
0,"Braund, Mr. Owen Harris",1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,"Heikkinen, Miss. Laina",0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"Allen, Mr. William Henry",0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,"Rice, Mrs. William (Margaret Norton)",0,1,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
708,"Montvila, Rev. Juozas",0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
709,"Graham, Miss. Margaret Edith",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
710,"Behr, Mr. Karl Howell",0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Modeling

In [40]:
# Train-test split
X_train1, X_test1, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=42)

In [41]:
X_train1.columns

Index(['Name', 'Siblings and Spouses', 'female', 'Cabin Class 2',
       'Cabin Class 3', 'Cherbourg', 'Queenstown', 'Age (20, 28]',
       'Age (28, 38]', 'Age (38, 80]', 'Mrs.', 'Miss', 'VIP', 'Fare (0, 10]',
       'Fare (10, 20]', 'Fare (20, 30]', 'Fare (30, 40]', 'Fare (40, 50]',
       'Fare (50, 60]', 'Fare (60, 70]', 'Fare (70, 80]', 'Fare (80, 90]',
       'Fare (90, 100]', 'Fare (100, 110]', 'Fare (110, 120]',
       'Fare (120, 140]', 'Fare (140, 150]', 'Fare (150, 160]',
       'Fare (160, 170]', 'Fare (170, 220]', 'Fare (220, 240]',
       'Fare (240, 250]', 'Fare (250, 270]', 'Fare (270, 600]'],
      dtype='object')

In [43]:
# Remove the names, we'll save those for later use.
X_train=X_train1.drop('Name', axis=1)
X_test=X_test1.drop('Name', axis=1)

In [44]:
X_train.columns

Index(['Siblings and Spouses', 'female', 'Cabin Class 2', 'Cabin Class 3',
       'Cherbourg', 'Queenstown', 'Age (20, 28]', 'Age (28, 38]',
       'Age (38, 80]', 'Mrs.', 'Miss', 'VIP', 'Fare (0, 10]', 'Fare (10, 20]',
       'Fare (20, 30]', 'Fare (30, 40]', 'Fare (40, 50]', 'Fare (50, 60]',
       'Fare (60, 70]', 'Fare (70, 80]', 'Fare (80, 90]', 'Fare (90, 100]',
       'Fare (100, 110]', 'Fare (110, 120]', 'Fare (120, 140]',
       'Fare (140, 150]', 'Fare (150, 160]', 'Fare (160, 170]',
       'Fare (170, 220]', 'Fare (220, 240]', 'Fare (240, 250]',
       'Fare (250, 270]', 'Fare (270, 600]'],
      dtype='object')

In [45]:
gnb = GaussianNB()
# Fit on the training data
gnb_model = gnb.fit(X_train, y_train)
# Predict on the testing data
predictions=gnb_model.predict(X_test)
probabilities = gnb_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_nb=metrics.roc_auc_score(y_test, predictions)
acc_nb = metrics.accuracy_score(y_test, predictions)
f1_nb = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_nb,4))
print('Accuracy', "%.4f" % round(acc_nb,4))
print('AUC Score', "%.4f" % round(auc_nb,4))

F1 Score 0.2821
Accuracy 0.6084
AUC Score 0.5623


In [46]:
knn = KNeighborsClassifier(n_neighbors=7)
# Fit on the training data
knn_model=knn.fit(X_train, y_train)
# Predict on the testing data
predictions=knn_model.predict(X_test)
probabilities = knn_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_knn=metrics.roc_auc_score(y_test, predictions)
acc_knn = metrics.accuracy_score(y_test, predictions)
f1_knn = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_knn,4))
print('Accuracy', "%.4f" % round(acc_knn,4))
print('AUC Score', "%.4f" % round(auc_knn,4))

F1 Score 0.6609
Accuracy 0.7273
AUC Score 0.7141


In [122]:
rf = RandomForestClassifier(max_depth=10)
# Fit on the training data
rf_model=rf.fit(X_train, y_train)
# Predict on the testing data
predictions=rf_model.predict(X_test)
probabilities = rf_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_rf=metrics.roc_auc_score(y_test, predictions)
acc_rf = metrics.accuracy_score(y_test, predictions)
f1_rf = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_rf,4))
print('Accuracy', "%.4f" % round(acc_rf,4))
print('AUC Score', "%.4f" % round(auc_rf,4))

F1 Score 0.7156
Accuracy 0.7832
AUC Score 0.7658


In [68]:
! pip install  graphviz

Looking in indexes: https://artifact.intuit.com/artifactory/api/pypi/pypi-intuit/simple
     |████████████████████████████████| 46 kB 562 kB/s eta 0:00:011
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.


In [48]:
logreg = LogisticRegression()
# Fit on the training data
log_model=logreg.fit(X_train, y_train)
# Predict on the testing data
predictions=log_model.predict(X_test)
probabilities = log_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_log=metrics.roc_auc_score(y_test, predictions)
acc_log = metrics.accuracy_score(y_test, predictions)
f1_log = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_log,4))
print('Accuracy', "%.4f" % round(acc_log,4))
print('AUC Score', "%.4f" % round(auc_log,4))

F1 Score 0.7273
Accuracy 0.7692
AUC Score 0.7617


### Comparison of Four Models

In [74]:
# create lists from the metrics we produced.
f1=[f1_nb, f1_log, f1_knn, f1_rf]
acc=[acc_nb, acc_log, acc_knn, acc_rf]
auc=[auc_nb, auc_log, auc_knn, auc_rf]
# Define a function that will round our metrics.
def rounder(metric):
    scores_list=[]
    for score in metric:
        scores_list.append(round(float(score*100),1))
    return scores_list
# Apply it to each of the three lists.
f1_scores=rounder(f1)
acc_scores=rounder(acc)
auc_scores=rounder(auc)
score_types=['F1 score', 'Accuracy', 'AUC score']

In [75]:
# Comparison of model metrics
models=['naive bayes', 'logistic regression', 'k-nearest neighbors', 'random forest']
index=['F1 score', 'Accuracy', 'AUC score']
compare_models=pd.DataFrame([f1_scores, acc_scores, auc_scores], index=index, columns=models)
compare_models

,naive bayes,logistic regression,k-nearest neighbors,random forest
F1 score,28.2,72.7,66.1,67.9
Accuracy,60.8,76.9,72.7,76.2
AUC score,56.2,76.2,71.4,74.2


In [76]:
# save to csv, for later use by plotly dash app.
compare_models.to_csv('resources/compare_models.csv', index=True)
pd.read_csv('resources/compare_models.csv', index_col=0)

,naive bayes,logistic regression,k-nearest neighbors,random forest
F1 score,28.2,72.7,66.1,67.9
Accuracy,60.8,76.9,72.7,76.2
AUC score,56.2,76.2,71.4,74.2


In [124]:
# Let's display that with plotly.
mydata1 = go.Bar(
    x=compare_models.loc['F1 score'].index,
    y=compare_models.loc['F1 score'],
    name=compare_models.index[0],
    marker=dict(color=Viridis[50])
)
mydata2 = go.Bar(
    x=compare_models.loc['Accuracy'].index,
    y=compare_models.loc['Accuracy'],
    name=compare_models.index[1],
    marker=dict(color=Viridis[30])
)
mydata3 = go.Bar(
    x=compare_models.loc['AUC score'].index,
    y=compare_models.loc['AUC score'],
    name=compare_models.index[2],
    marker=dict(color=Viridis[10])
)
mylayout = go.Layout(
    title='Comparison of Possible Models',
    xaxis = dict(title = 'Predictive models'), # x-axis label
    yaxis = dict(title = 'Score'), # y-axis label
    
)
fig = go.Figure(data=[mydata1, mydata2, mydata3], layout=mylayout)
fig

In [125]:
file = open('resources/final_random_model.pkl', 'wb')
pickle.dump(rf_model, file)
file.close()

## Tuning the Logistic Classifier
Note: The gridsearch step is included here for completeness sake, as this is a smart inclusion in any iteration of possible models. But for the sake of speed (this notebook was run multiple times during development) I've kept my gridsearch to a bare-bones placeholder. A more complete project would use a broader grid.

In [87]:
# Create regularization penalty space (l1=ridge, l2=lasso)
penalty = ['l1', 'l2'] 

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty, max_iter=[3000])

# Create grid search using 5-fold cross validation
grid_lr = GridSearchCV(LogisticRegression(), hyperparameters, cv=5,  n_jobs = 1, verbose=0)
#grid_lr = GridSearchCV(RandomForestClassifier(), hyperparameters, cv=5,  n_jobs = 1, verbose=0)
grid_lr.fit(X_train, y_train)

ValueError: Invalid parameter C for estimator RandomForestClassifier(). Check the list of available parameters with `estimator.get_params().keys()`.

In [82]:
print(grid_lr.best_params_)

log_model = grid_lr

{'C': 21.544346900318832, 'max_iter': 6000, 'penalty': 'l2'}


In [91]:
# Predict on the testing data
load_model = rf_model
predictions=log_model.predict(X_test)
probabilities = log_model.predict_proba(X_test)[:,1]
print('text finished')
predictions

text finished


array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [56]:
# Pickle the final model for use in the plotly dash app.
file = open('resources/final_logreg_model_modified.pkl', 'wb')
pickle.dump(log_model, file)
file.close()

## Final Model Metrics

In [123]:
# Full list of metrics
def model_metrics(y_test, predictions):
    '''
    Calculate 5 standard model metrics
    Return a dictionary with the metrics
    '''
    f1 = metrics.f1_score(y_test, predictions)
    accuracy = metrics.accuracy_score(y_test, predictions)
    error = 1 - accuracy
    precision = metrics.precision_score(y_test, predictions)
    recall = metrics.recall_score(y_test, predictions)
    rocauc =  metrics.roc_auc_score(y_test, predictions)
    return {'precision': precision, 'recall': recall,'f1 score':f1, 'accuracy': accuracy, 'error rate': error,  'ROC-AUC': rocauc}

eval_scores=model_metrics(y_test, predictions)
eval_scores

{'precision': 0.8478260869565217,
 'recall': 0.6190476190476191,
 'f1 score': 0.7155963302752294,
 'accuracy': 0.7832167832167832,
 'error rate': 0.21678321678321677,
 'ROC-AUC': 0.7657738095238096}

In [126]:
# Round the y values.
y_vals=[]
for val in list(eval_scores.values()):
    y_vals.append(round(float(val*100),1))
y_vals    
# Write over the previous dictionary with the rounded values.
eval_scores=dict(zip(eval_scores.keys(), y_vals))
print(eval_scores)

{'precision': 84.8, 'recall': 61.9, 'f1 score': 71.6, 'accuracy': 78.3, 'error rate': 21.7, 'ROC-AUC': 76.6}


In [127]:
# Now save that dictionary to a pickle file, for later use in plotly dash app
file = open('resources/eval_scores.pkl', 'wb')
pickle.dump(eval_scores, file)
file.close()

In [128]:
# And here's a reminder of how to read that back in again, just in case this is unfamiliar:
file = open('resources/eval_scores.pkl', 'rb')
evals=pickle.load(file)
file.close()
evals

{'precision': 84.8,
 'recall': 61.9,
 'f1 score': 71.6,
 'accuracy': 78.3,
 'error rate': 21.7,
 'ROC-AUC': 76.6}

In [40]:
# Convert that into a visualization.
mydata = [go.Bar(
    x=list(evals.keys()),
    y=list(evals.values()),
    marker=dict(color=Viridis[::12])
)]

mylayout = go.Layout(
    title='Evaluation Metrics for Logistic Regression Model (Testing Dataset = 127 passengers)',
    xaxis = {'title': 'Metrics'},
    yaxis = {'title': 'Percent'}, 

)
fig = go.Figure(data=mydata, layout=mylayout)
fig

In [129]:
FPR, TPR, _ = roc_curve(y_test, probabilities)
FPR

array([0.    , 0.    , 0.    , 0.0125, 0.0125, 0.0125, 0.0125, 0.025 ,
       0.025 , 0.0375, 0.0375, 0.05  , 0.075 , 0.0875, 0.0875, 0.1   ,
       0.1125, 0.1125, 0.125 , 0.1625, 0.1625, 0.1875, 0.1875, 0.25  ,
       0.25  , 0.25  , 0.25  , 0.3   , 0.3   , 0.3375, 0.3625, 0.375 ,
       0.4   , 0.425 , 0.45  , 0.4625, 0.4625, 0.4875, 0.525 , 0.525 ,
       0.55  , 0.5875, 0.6   , 0.75  , 0.8125, 0.825 , 0.825 , 0.875 ,
       0.875 , 0.925 , 0.925 , 0.95  , 1.    ])

In [130]:
roc_score=round(100*roc_auc_score(y_test, predictions),1)
roc_score

76.6

In [131]:
roc_dict={'FPR':list(FPR), 
          'TPR':list(TPR),
          'y_test':list(y_test), 
          'predictions':[int(i) for i in predictions]
         }

In [132]:
# Save everything we need to reproduce the ROC-AUC figure in plotly dash.
import json
with open('resources/roc_dict.json', 'w') as f:
    json.dump(roc_dict, f)

In [133]:
with open('resources/roc_dict.json') as json_file:
    roc_dict = json.load(json_file)
FPR=roc_dict['FPR']
TPR=roc_dict['TPR']
y_test=pd.Series(roc_dict['y_test'])
predictions=roc_dict['predictions']

In [104]:
# # Save everything we need to reproduce the ROC-AUC figure in plotly dash.
import json
file = open('resources/FPR.json', 'wb')
json.dump(FPR, file)
file.close()

file = open('resources/TPR.pkl', 'wb')
pickle.dump(TPR, file)
file.close()

file = open('resources/y_test.pkl', 'wb')
pickle.dump(y_test, file)
file.close()

file = open('resources/predictions.pkl', 'wb')
pickle.dump(predictions, file)
file.close()

TypeError: a bytes-like object is required, not 'str'

In [134]:
# ROC-AUC figure

roc_score=round(100*roc_auc_score(y_test, predictions),1)
trace0=go.Scatter(
        x=FPR, 
        y=TPR,
        mode='lines',
        name=f'AUC: {roc_score}',
        marker=dict(color=Viridis[10])
        )
trace1=go.Scatter(
        x=[0,1], 
        y=[0,1],
        mode='lines',
        name='Baseline Area: 50.0',
    marker=dict(color=Viridis[50])
        )
layout=go.Layout(
    title='Receiver Operating Characteristic (ROC): Area Under Curve',
    xaxis={'title': 'False Positive Rate (100-Specificity)','scaleratio': 1,'scaleanchor': 'y'},
    yaxis={'title': 'True Positive Rate (Sensitivity)'}
    )
data=[trace0, trace1]
fig = dict(data=data, layout=layout)
fig

{'data': [Scatter({
      'marker': {'color': '#46307c'},
      'mode': 'lines',
      'name': 'AUC: 76.6',
      'x': [0.0, 0.0, 0.0, 0.0125, 0.0125, 0.0125, 0.0125, 0.025, 0.025, 0.0375,
            0.0375, 0.05, 0.075, 0.0875, 0.0875, 0.1, 0.1125, 0.1125, 0.125, 0.1625,
            0.1625, 0.1875, 0.1875, 0.25, 0.25, 0.25, 0.25, 0.3, 0.3, 0.3375, 0.3625,
            0.375, 0.4, 0.425, 0.45, 0.4625, 0.4625, 0.4875, 0.525, 0.525, 0.55,
            0.5875, 0.6, 0.75, 0.8125, 0.825, 0.825, 0.875, 0.875, 0.925, 0.925,
            0.95, 1.0],
      'y': [0.0, 0.015873015873015872, 0.12698412698412698, 0.14285714285714285,
            0.25396825396825395, 0.2857142857142857, 0.3492063492063492,
            0.3492063492063492, 0.47619047619047616, 0.47619047619047616,
            0.5079365079365079, 0.5079365079365079, 0.5396825396825397,
            0.5396825396825397, 0.6190476190476191, 0.6190476190476191,
            0.6349206349206349, 0.6666666666666666, 0.6825396825396826,
          

In [135]:
# A confusion matrix tells us our false positives and false negatives:
matrix=confusion_matrix(y_test, predictions)
print(matrix)
cm=pd.DataFrame(matrix, columns=['pred: survival', 'pred: death'])
cm[f'n={len(y_test)}']=['actual: survival', 'actual: death']
cm=cm[[f'n={len(y_test)}', 'pred: survival', 'pred: death']]
cm

[[73  7]
 [24 39]]


,n=143,pred: survival,pred: death
0,actual: survival,73,7
1,actual: death,24,39


In [136]:
# Save cm dataframe to a pickle file, for later use in plotly dash app
cm.to_csv('resources/confusion_matrix.csv', index=False)
cm=pd.read_csv('resources/confusion_matrix.csv')
cm

,n=143,pred: survival,pred: death
0,actual: survival,73,7
1,actual: death,24,39


In [137]:
# Display the confusion matrix as a formatted table with Plotly
trace = go.Table(
    header=dict(values=cm.columns,
                line = dict(color='#7D7F80'),
                fill = dict(color=Viridis[55]),
                align = ['left'] * 5),
    cells=dict(values=[cm[f'n={len(y_test)}'], cm['pred: survival'], cm['pred: death']],
               line = dict(color='#7D7F80'),
               fill = dict(color='white'),
               align = ['left'] * 5))

layout = go.Layout(
    title = f'Confusion Matrix: Logistic Regression Model (Testing Dataset)'
)

data = [trace]
fig = dict(data=data, layout=layout)
fig

{'data': [Table({
      'cells': {'align': [left, left, left, left, left],
                'fill': {'color': 'white'},
                'line': {'color': '#7D7F80'},
                'values': [['actual: survival', 'actual: death'], [73, 24], [7, 39]]},
      'header': {'align': [left, left, left, left, left],
                 'fill': {'color': '#6bcc5a'},
                 'line': {'color': '#7D7F80'},
                 'values': array(['n=143', 'pred: survival', 'pred: death'], dtype=object)}
  })],
 'layout': Layout({
     'title': {'text': 'Confusion Matrix: Logistic Regression Model (Testing Dataset)'}
 })}

In [109]:
# Feature importance (Logistic Regression)
coeffs1=pd.DataFrame(list(zip(list(X_train.columns), logreg.coef_[0])), columns=['feature', 'coefficient'])
coeffs=coeffs1.sort_values(by='coefficient', ascending=False)

# Format the coefficients.
y_vals=[]
for val in list(coeffs['coefficient']):
    y_vals.append(round(float(val),2))
y_vals

coeffs['coefficient']=y_vals
coeffs

,feature,coefficient
1,female,2.03
9,Mrs.,1.10
17,"Fare (50, 60]",0.83
21,"Fare (90, 100]",0.54
4,Cherbourg,0.46
24,"Fare (120, 140]",0.34
20,"Fare (80, 90]",0.33
10,Miss,0.33
23,"Fare (110, 120]",0.28
27,"Fare (160, 170]",0.19


In [138]:
# save the results to a csv file, for later use by plotly dash app.
coeffs.to_csv('resources/coefficients.csv', index=False)

In [139]:
# Let's display that with Plotly.
mydata = [go.Bar(
    x=coeffs['feature'],
    y=coeffs['coefficient'],
    marker=dict(color=Viridis[::-6])
)]

mylayout = go.Layout(
    title='Married women in 1st class had better odds of survival, especially if younger than 38',
    xaxis = {'title': 'Passenger Features'},
    yaxis = {'title': 'Odds of Survival'}, 

)
fig = go.Figure(data=mydata, layout=mylayout)
fig

In [140]:
print(len(probabilities))
print(len(predictions))
print(len(y_test))
print(len(X_test1))

143
143
143
143


In [141]:
X_test1=X_test1.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)
probs=pd.DataFrame(probabilities, columns=['survival_prob'])

In [142]:
# Merge back in the names
final=pd.concat([X_test1, y_test, probs], axis=1)
final.rename(columns={0: "Survived"},inplace=True)

In [143]:
final.shape

(143, 36)

In [144]:
final.head()

,Name,Siblings and Spouses,female,Cabin Class 2,Cabin Class 3,Cherbourg,Queenstown,"Age (20, 28]","Age (28, 38]","Age (38, 80]",...,"Fare (140, 150]","Fare (150, 160]","Fare (160, 170]","Fare (170, 220]","Fare (220, 240]","Fare (240, 250]","Fare (250, 270]","Fare (270, 600]",Survived,survival_prob
0,"Sagesser, Mlle. Emma",0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0.947415
1,"Eustis, Miss. Elizabeth Mussey",1,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0.922534
2,"Taussig, Mr. Emil",1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.402966
3,"Ryerson, Miss. Emily Borie",2,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0.941133
4,"Sharp, Mr. Percival James R",0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.398675


In [145]:
final.to_csv('resources/final_probs.csv', index=False)

In [146]:
list(final.columns)

['Name',
 'Siblings and Spouses',
 'female',
 'Cabin Class 2',
 'Cabin Class 3',
 'Cherbourg',
 'Queenstown',
 'Age (20, 28]',
 'Age (28, 38]',
 'Age (38, 80]',
 'Mrs.',
 'Miss',
 'VIP',
 'Fare (0, 10]',
 'Fare (10, 20]',
 'Fare (20, 30]',
 'Fare (30, 40]',
 'Fare (40, 50]',
 'Fare (50, 60]',
 'Fare (60, 70]',
 'Fare (70, 80]',
 'Fare (80, 90]',
 'Fare (90, 100]',
 'Fare (100, 110]',
 'Fare (110, 120]',
 'Fare (120, 140]',
 'Fare (140, 150]',
 'Fare (150, 160]',
 'Fare (160, 170]',
 'Fare (170, 220]',
 'Fare (220, 240]',
 'Fare (240, 250]',
 'Fare (250, 270]',
 'Fare (270, 600]',
 'Survived',
 'survival_prob']

In [147]:
mydata=final.drop(['survival_prob'], axis=1)
table=[go.Table(
        header=dict(values=list(mydata.columns)),
        cells=dict(values=list(mydata.loc[5])))]
table

[Table({
     'cells': {'values': [Thayer, Mr. John Borland Jr, 0, 0, 0, 0, 1, 0, 0, 0, 0,
                          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
                          0, 0, 0, 0, 0, 1]},
     'header': {'values': [Name, Siblings and Spouses, female, Cabin Class 2, Cabin
                           Class 3, Cherbourg, Queenstown, Age (20, 28], Age (28,
                           38], Age (38, 80], Mrs., Miss, VIP, Fare (0, 10], Fare
                           (10, 20], Fare (20, 30], Fare (30, 40], Fare (40, 50],
                           Fare (50, 60], Fare (60, 70], Fare (70, 80], Fare (80,
                           90], Fare (90, 100], Fare (100, 110], Fare (110, 120],
                           Fare (120, 140], Fare (140, 150], Fare (150, 160], Fare
                           (160, 170], Fare (170, 220], Fare (220, 240], Fare (240,
                           250], Fare (250, 270], Fare (270, 600], Survived]}
 })]

## Exploring individual predictions

In [63]:
value='Sharp, Mr. Percival James R'
survival=final.loc[3, 'survival_prob']
survival

0.9372653748980653

In [64]:
names=df['Name'].values
names[:5]

array(['Braund, Mr. Owen Harris',
       'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
       'Heikkinen, Miss. Laina',
       'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
       'Allen, Mr. William Henry'], dtype=object)

In [65]:
indexs=df['Name'].index.values
indexs[:5]

array([0, 1, 2, 3, 4])

In [66]:
names=df['Name'].values
index=df['Name'].index.values
nameslist = list(zip(indexs, names))
print(nameslist[5])
print(nameslist[5][0])
print(nameslist[5][1])

(5, 'McCarthy, Mr. Timothy J')
5
McCarthy, Mr. Timothy J


In [67]:
options=[{'label': k, 'value': i} for i,k in nameslist]
options[0]

{'label': 'Braund, Mr. Owen Harris', 'value': 0}

In [68]:
value=nameslist[0][0]
value

0

In [69]:
survival=final.loc[value, 'survival_prob']
round(survival*100)

94

In [70]:
final.columns

Index(['Name', 'Siblings and Spouses', 'female', 'Cabin Class 2',
       'Cabin Class 3', 'Cherbourg', 'Queenstown', 'Age (20, 28]',
       'Age (28, 38]', 'Age (38, 80]', 'Mrs.', 'Miss', 'VIP', 'Survived',
       'survival_prob'],
      dtype='object')

## Predict on a single, individual row of data.

In [71]:
final.head()

,Name,Siblings and Spouses,female,Cabin Class 2,Cabin Class 3,Cherbourg,Queenstown,"Age (20, 28]","Age (28, 38]","Age (38, 80]",Mrs.,Miss,VIP,Survived,survival_prob
0,"Sagesser, Mlle. Emma",0,1,0,0,1,0,1,0,0,0,1,0,1,0.938691
1,"Eustis, Miss. Elizabeth Mussey",1,1,0,0,1,0,0,0,1,0,1,0,1,0.847526
2,"Taussig, Mr. Emil",1,0,0,0,0,0,0,0,1,0,0,0,0,0.276496
3,"Ryerson, Miss. Emily Borie",2,1,0,0,1,0,0,0,0,0,1,0,1,0.937265
4,"Sharp, Mr. Percival James R",0,0,1,0,0,0,1,0,0,0,0,0,0,0.231061


In [72]:
testset=final.drop(["survival_prob", 'Name','Survived'], axis=1)
firstrow=testset.loc[0]
firstrow

Siblings and Spouses    0
female                  1
Cabin Class 2           0
Cabin Class 3           0
Cherbourg               1
Queenstown              0
Age (20, 28]            1
Age (28, 38]            0
Age (38, 80]            0
Mrs.                    0
Miss                    1
VIP                     0
Name: 0, dtype: int64

In [73]:
myarray=firstrow.values
myarray.shape

(12,)

In [74]:
thisarray=myarray.reshape((1, myarray.shape[0]))
thisarray.shape

(1, 12)

In [75]:
logreg.predict_proba(thisarray)

array([[0.07385792, 0.92614208]])

In [76]:
logreg.predict(thisarray)

array([1])

In [77]:
df.columns

Index(['Survived', 'Age', 'Fare', 'Name', 'Siblings and Spouses', 'female',
       'male', 'Cabin Class 1', 'Cabin Class 2', 'Cabin Class 3', 'Cherbourg',
       'Queenstown', 'Southampton', 'Age (1, 20]', 'Age (20, 28]',
       'Age (28, 38]', 'Age (38, 80]', 'Last Name', 'First Name', 'Title',
       'Mr.', 'Mrs.', 'Miss', 'VIP'],
      dtype='object')